In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm  # 진행 상황을 시각적으로 표시하기 위한 라이브러리

# EXAONE 모델과 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

# CSV 파일 로드
input_csv_path = 'noise.csv'  # 입력 CSV 파일 경로
df = pd.read_csv(input_csv_path)

# 결과를 저장할 리스트
cleaned_texts = []

# tqdm을 사용하여 진행 상황을 표시
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Texts"):
    noisy_text = row['text']  # 'text'는 노이즈가 포함된 컬럼 이름입니다.

    # 노이즈 제거를 위한 프롬프트 생성
    prompt = f"다음 텍스트에서 불필요한 특수문자나 잡음을 제거하고 정제된 문장으로 수정하세요: '{noisy_text}'"

    # 메시지 형식화
    messages = [
        {"role": "system", 
         "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    # 입력 텐서 준비
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )

    # 모델에서 텍스트 생성 (노이즈 제거)
    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=128
    )

    # 출력 디코딩 및 정제된 텍스트 추출 (프롬프트 부분을 제외하고 실제 정제된 텍스트만 추출)
    cleaned_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # 정제된 텍스트에서 프롬프트 부분을 제외한 텍스트만 추출
    start_idx = cleaned_text.find("다음은 불필요한 특수문자와 잡음을 제거한 정제된 문장입니다:")
    if start_idx != -1:
        cleaned_text = cleaned_text[start_idx + len("다음은 불필요한 특수문자와 잡음을 제거한 정제된 문장입니다:"):].strip()

    # 결과 리스트에 추가
    cleaned_texts.append(cleaned_text)

# 정제된 텍스트를 데이터프레임에 추가
df['cleaned_text'] = cleaned_texts

# 결과 CSV 파일로 저장
output_csv_path = 'cleaned_texts.csv'
df.to_csv(output_csv_path, index=False)

print(f"노이즈가 제거된 텍스트가 {output_csv_path}에 저장되었습니다.")


In [5]:
import pandas as pd
import re

def process_cleaned_text(file_path):
    # CSV 파일을 읽어들입니다.
    df = pd.read_csv(file_path)
    
    # 'cleaned_text' 열을 처리합니다.
    def process_text(text):
        # 텍스트에서 모든 따옴표 내부의 내용들을 찾아 리스트로 저장합니다.
        matches = re.findall(r"['\"](.*?)['\"]", text)
        
        if matches:
            # 두 번째 따옴표 내부의 내용을 반환 (인덱스 1)
            if len(matches) > 1:
                return matches[1]
            # 하나만 있을 경우 그 하나를 반환
            return matches[0]
        else:
            # 따옴표가 없으면 첫 번째 줄만 반환
            return text.split('\n')[0]
    
    # 'cleaned_text' 열에 적용
    df['augmented_title'] = df['augmented_title'].apply(process_text)
    
    # 'cleaned_text' 열을 'text' 열로 변경
    df['text'] = df['augmented_title']
    
    # 기존 'text' 열 삭제
    df.drop(columns=['augmented_title'], inplace=True)
    
    # 변경된 DataFrame을 반환
    return df

# 파일 경로를 지정하여 호출
file_path = 'train_aug_aya.csv'  # 실제 파일 경로로 수정 필요
processed_df = process_cleaned_text(file_path)

# 처리된 데이터프레임 확인 (예시)
print(processed_df.head())

# 'text' 열에서 '증강된 제목: ' 문자열을 제거
processed_df['text'] = processed_df['text'].str.replace('증강된 제목: ', '', regex=False)

# 'text' 열에서 길이가 4글자 이하인 행 제거
processed_df = processed_df[processed_df['text'].str.len() > 4]

# 처리된 결과를 새로운 CSV 파일로 저장 (선택 사항)
processed_df.to_csv('exaone_clean_noise.csv', index=False)


                    ID                                               text  \
0  ynat-v1_train_00624                                                  실   
1  ynat-v1_train_02413                 전 세계가 우려하는 개조: 윤리적 경계에서 벗어난 기술의 미래   
2  ynat-v1_train_01203         미국 대통령 선거: 새로운 지도자, 새로운 시대를 향하여 워싱턴으로의 귀환    
3  ynat-v1_train_02105  주어진 뉴스 기사 제목은 이해하기 어려운 형태로 보입니다. 이를 바탕으로 유사한 제...   
4  ynat-v1_train_02765                   증강된 제목: 서울 관광객 증가, 도시 경제에 긍정적 영향   

   target  
0       0  
1       6  
2       2  
3       0  
4       3  


In [ ]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from tqdm import tqdm
import re

data = pd.read_csv('./data/noise.csv')

model = OllamaLLM(
    model='jmpark333/exaone',
    temperature=0.7,
    seed=42
)

def clean_text(text):
    cleaned = re.sub(r'[\[\]"]', '', text).strip()
    return cleaned

template = """
    다음은 뉴스 기사 제목입니다:
    뉴스 기사 제목: "{Text}"
    
    지시사항:
    1. 뉴스 기사 제목을 읽어.
    2. 유사한 내용의 제목을 하나 생성해.
    
    지정 답변 형식: "[증강된 제목]"
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

augmented_texts = []
for text in tqdm(data['text'].to_list(), desc='데이터 증강 중...', total=len(data)):
    response = chain.invoke({"Text": text})
    cleaned_response = clean_text(response)
    augmented_texts.append(cleaned_response)

data['augmented_text'] = augmented_texts
data.to_csv('train_aug_exaone.csv', index=False)


In [1]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from tqdm import tqdm
import re

# 데이터 로드
data = pd.read_csv('./data/noise.csv')

# 모델 설정
model = OllamaLLM(
    model='aya-expanse:8b',  # 모델 변경 필요
    temperature=0.7,  # 다양성 조절
    seed=42
)

def clean_text(text):
    # 텍스트에서 불필요한 대괄호와 따옴표를 제거하고 공백을 정리
    cleaned = re.sub(r'[\[\]"]', '', text).strip()
    return cleaned

# 증강할 텍스트 형식 정의 (주제 변경 또는 유사 제목 생성)
template = """
    다음은 뉴스 기사 제목입니다.
    뉴스 기사 제목: "{Text}"
    지시사항:
    1. 주어진 뉴스 기사 제목을 읽어라.
    2. 해당 제목을 바탕으로 유사한 제목을 생성하라.
    지정 답변 형식: "[증강된 제목]"
"""

# 템플릿 생성 및 체인 설정
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model

# 데이터 증강
augmented_titles = []
for text in tqdm(data['text'].to_list(), desc='증강 중...', total=len(data)):
    # 모델에 입력하여 응답 받기
    response = chain.invoke({"Text": text})
    
    # 응답에서 불필요한 부분 제거
    cleaned_response = clean_text(response)
    augmented_titles.append(cleaned_response)

# 증강된 제목을 원본 데이터에 추가
data['augmented_title'] = augmented_titles

# 결과 CSV 파일로 저장
data.to_csv('train_aug_aya.csv', index=False)

print("증강된 제목이 'train_augmented_titles.csv'에 저장되었습니다.")


증강 중...: 100%|██████████| 1595/1595 [34:44<00:00,  1.31s/it]

증강된 제목이 'train_augmented_titles.csv'에 저장되었습니다.


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 데이터 로드
df = pd.read_csv("noise_free.csv")  # noise_free.csv 경로를 정확히 지정하세요
texts = df['text'].values  # 텍스트 데이터

# 2. KoBERT 모델과 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModel.from_pretrained("monologg/kobert")

# 3. 텍스트를 KoBERT로 임베딩
def embed_text(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
        # 평균값을 구하여 텍스트 임베딩으로 사용
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        embeddings.append(embedding)
    return embeddings

# 임베딩 생성
embeddings = embed_text(texts)

# 4. KMeans 클러스터링
num_clusters = 7  # 7개의 클러스터로 분류
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(embeddings)

# 5. 클러스터 예측 결과
df['cluster'] = kmeans.labels_

# 6. 각 클러스터의 주요 단어 출력 (임베딩을 직접 해석하기 어려운 점을 고려하여 생략)
# 대신 각 클러스터별로 제목을 출력하여 확인

for i in range(num_clusters):
    cluster_titles = df[df['cluster'] == i]['text'].tolist()
    print(f"Cluster {i}:")
    print("\n".join(cluster_titles[:5]))  # 각 클러스터의 상위 5개 뉴스 제목 출력
    print()

# 7. 클러스터링 결과 확인 (선택 사항)
plt.figure(figsize=(8, 6))
sns.countplot(x='cluster', data=df)
plt.title("Cluster Distribution")
plt.xlabel("Cluster")
plt.ylabel("Count")
plt.show()

# 8. 결과 파일로 저장 (선택 사항)
df.to_csv("clustered_noise_free.csv", index=False)